In [1]:
import json
import pickle
import random

import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.nn.utils.rnn as rnn_utils

import nltk
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
import time

from Vocab import Vocab

import torch
torch.cuda.set_device(0)

print('import over')

import over


In [45]:
def batch_tokens_bleu(references, candidates, smooth_epsilon=0.001):
    ##    para: references and candidates are list[list] type
    ##    return: list of BLEU for every sample
    ##    
    bleu_scores=[]
    for ref, candidate in zip(references, candidates):
        candidate_len = len(candidate)
        if candidate_len==0:
            bleu_scores.append(0)
        elif candidate_len==1:
            bleu_scores.append(sentence_bleu([ref], candidate, weights=[1]))
        elif candidate_len==2:
            bleu_scores.append(sentence_bleu([ref], candidate, weights=[0.5,0.5]))
        elif candidate_len==3:
            bleu_scores.append(sentence_bleu([ref], candidate, weights=[0.3333,0.3333,0.3333]))
        else:
            bleu_scores.append(sentence_bleu([ref], candidate, smoothing_function = SmoothingFunction(epsilon=smooth_epsilon).method1))
    return bleu_scores


a=[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
b=[[1], [1,'a'], [1,2,3], [1,2,3,'a']]
print(batch_tokens_bleu(a, b))
print(sentence_bleu([a[0]], [1,2], weights=[0.5,0.5]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,'a','b','c','d','e'], weights=[0.5,0.5]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,3,'a','b','c','d'], weights=[1/3,1/3,1/3]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,3,4,'a','b','c'], weights=[0.25, 0.25, 0.25, 0.25]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))

print(sentence_bleu([a[0]], [1,'a','b','c','d','e','f'], smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,'a','b','c','d','e'], smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,3,'a','b','c','d'],  smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,3,4,'a','b','c'], weights=[0.25, 0.25, 0.25, 0.25], smoothing_function = SmoothingFunction(epsilon=0.001).method1))

print(sentence_bleu([a[0]], [1,'a','b','c','d','e','f'], weights=[1]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,'a'], weights=[0.5,0.5]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,3,'a','b'], weights=[1/3,1/3,1/3]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))
print(sentence_bleu([a[0]], [1,2,3,4], weights=[0.25, 0.25, 0.25, 0.25]))#, smoothing_function = SmoothingFunction(epsilon=0.001).method1))


[5.602796437537268e-09, 8.726390933459443e-05, 0.0034593773364647584, 0.002302494254638862]
0.00012340980408667956
0.03406775047661826
0.04772695827953738
0.06418537663419444
0.0005377174819278786
0.0023010355994352106
0.011402567865604762
0.06418537663419444
0.02230257790228158
0.0019972724364364187
0.02310911005945933
0.01831563888873418


In [9]:
def _inflate(tensor, times, dim):
    """
    Examples::
        >> a = torch.LongTensor([[1, 2], [3, 4]])
        >> a
        1   2
        3   4
        [torch.LongTensor of size 2x2]
        >> b = ._inflate(a, 2, dim=1)
        >> b
        1   2   1   2
        3   4   3   4
        [torch.LongTensor of size 2x4]
    """
    repeat_dims = [1] * tensor.dim()
    repeat_dims[dim] = times
    return tensor.repeat(*repeat_dims)

a=torch.rand(2,2)
print(a)
b=_inflate(a, times=3,dim=0)
print(b)


 0.3481  0.2966
 0.1024  0.0319
[torch.FloatTensor of size 2x2]


 0.3481  0.2966
 0.1024  0.0319
 0.3481  0.2966
 0.1024  0.0319
 0.3481  0.2966
 0.1024  0.0319
[torch.FloatTensor of size 6x2]



In [7]:
a=[torch.rand(4,3,1) for _ in range(2)]
print(a)
a=torch.cat(a, dim=2)
b=a.view(4*3, -1)
print(b)
c=b.view(4,3,-1)
print(c)

[
(0 ,.,.) = 
  0.5779
  0.7873
  0.1317

(1 ,.,.) = 
  0.4067
  0.1400
  0.0116

(2 ,.,.) = 
  0.7928
  0.0855
  0.4257

(3 ,.,.) = 
  0.9640
  0.4155
  0.8866
[torch.FloatTensor of size 4x3x1]
, 
(0 ,.,.) = 
  0.6335
  0.8259
  0.8924

(1 ,.,.) = 
  0.2542
  0.3721
  0.2569

(2 ,.,.) = 
  0.7518
  0.3747
  0.2418

(3 ,.,.) = 
  0.8554
  0.0451
  0.8294
[torch.FloatTensor of size 4x3x1]
]

 0.5779  0.6335
 0.7873  0.8259
 0.1317  0.8924
 0.4067  0.2542
 0.1400  0.3721
 0.0116  0.2569
 0.7928  0.7518
 0.0855  0.3747
 0.4257  0.2418
 0.9640  0.8554
 0.4155  0.0451
 0.8866  0.8294
[torch.FloatTensor of size 12x2]


(0 ,.,.) = 
  0.5779  0.6335
  0.7873  0.8259
  0.1317  0.8924

(1 ,.,.) = 
  0.4067  0.2542
  0.1400  0.3721
  0.0116  0.2569

(2 ,.,.) = 
  0.7928  0.7518
  0.0855  0.3747
  0.4257  0.2418

(3 ,.,.) = 
  0.9640  0.8554
  0.4155  0.0451
  0.8866  0.8294
[torch.FloatTensor of size 4x3x2]

